In [ ]:
# check_FP_output_loc.ipynb

# This is the precursor of the third programme of SOFT-IO-LI. It uses the outputs from the first two programmes to ascertain if the geographical
# location of the maximum residence times found by FLEXPART in programme 2 are over the Americas. At this time we only have access to cloud and
# lightning data in this region.
# If the maximum residence times are found elsewhere the data will be kept but not analysed any further at this time.

# Note, this programme only checks for the GOES16 satellite coverage because for the NOx flights from 2018 this is all that was available
# GOES16 coverage used -120° < longitude < -30° and -52° < latitude < 52°
""" Remarque: GOES coverage values en dur !! """

# C. Mackay March 2023 (Catherine.Mackay@aero.obs-mip.fr)
# https://github.com/ckmackay/SOFT-IO-LI.git

#Suggestions/improvements to be made:

# Add in coverage for GOES17 as required (any flights after 12-02-2019)*
# GOES17 coverage -180° < longitude < -90° and -52° < latitude < 52°
""" Remarque: GOES coverage values en dur !! """

""" Pistes d'amélioration : check for other satellite coverage (himawari, FY-4A, etc.) 
    --> définir zones pour chaque satellite et regarder si release dans une des zones définies ? 
    --> recup quelle zone OK pour aller chercher les données pour la suite 
    ? - on utilise des chunks et tout mais du coup faudrait pas initialiser un client dask ou jsp quoi ??
"""
# Write list of flights for further analysis to a txt file to find them easier later

In [1]:
import os
import os.path
from os.path import exists
from netCDF4 import Dataset
#from ncdump import ncdump
import numpy as np
import pandas as pd
import xarray as xr
import json

#for plotting
import hvplot.xarray # fancy plotting for xarray
import holoviews as hv
import matplotlib.pyplot as plt
import geoviews as gv

#for stats
import statistics
from statistics import mean, median, mode, stdev, median_high

#for datetime
import datetime
from datetime import datetime as dt
import matplotlib.dates as mdates
from matplotlib.dates import date2num

#for cartopy
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature

/home/patj/miniconda3/envs/softioli/lib/python3.10/site-packages/geoviews/operation/__init__.py:14: HoloviewsDeprecationWarning: 'ResamplingOperation' is deprecated and will be removed in version 1.17, use 'ResampleOperation2D' instead.
  from holoviews.operation.datashader import (


In [2]:
### Check the following paths and settings: these are set for SOFT-IO-LI default on nuwa in /o3p/iagos/SOFT-IO-LI
### For testing, think of creating /o3p/user/SOFT-IO-LI/ and changing the paths below. 
### you will need to create the following directories:
''' Remarques:
    - flight_path pas utilisé
'''
def set_paths():
    
    global flexpart_path, flight_path
    
    flexpart_path = '/o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/'
    flight_path = '/o3p/iagos/iagosv2/netcdf/L2/'
    

In [3]:
# find flights from directory

def find_flights():
    global flights
    flights=[]
    flights=sorted(os.listdir(flexpart_path))
    print(flights) ##############
    #print(len(flights))

In [9]:
# open netcdf file for flight
"""
<!> ouvre toujours le même fichier netcdf !!!!! parce que i=0
    --> OK now -> passe flight en param de la fonction
<!> check juste 1 seule release !! parce que return direct si OK pour GOES truc
    --> gérer cas où plusieurs releases à plusieurs endroits pour bien aller chercher les données satellite pour TOUTES les releases
        --> faudra associer qqpart chaque release à une zone géographique
<!> trop bizarre parce qu'en fait elle utilise pas spec_max après!! 
    et là regarde si bonne zone géo super bizarrement, PAS avec vrais coordonnées mais avec des sortes d'index ??
"""

def open_netcdf(flight):
    global da, netcdf_file
    ####### i=0
    for file in sorted(os.listdir(flexpart_path+flight+'/10j_100k_output/')):
        if file.endswith(".nc"):
            print("\nOpen file :",flexpart_path+flight+'/10j_100k_output/'+file)
            
            ds = xr.open_dataset(flexpart_path+flight+'/10j_100k_output/'+file)
            ds.spec001_mr.sizes
            ### ce truc (ds.spec001_mr) est rempli de 0 sauf à qqs endroits seulement j'imagine ?
            da = ds.spec001_mr.chunk({'nageclass': 1, 'pointspec': 1, 'time': 40, 'height': -1, 'latitude': -1, 'longitude': -1})
            # let's get rid of the auxilary dimension "nageclass", for the sake of clarity:
            da = da.squeeze(dim='nageclass')
            spec_argmax = da.argmax(dim=['longitude', 'latitude']) # returns index of max spec001_mr value
            # let's grab longitudes and latitudes of max. value of spec001_mr:
            lon_argmax = spec_argmax['longitude'] ### lon_argmax = dataArray [<release [<time> [<lon>]]>] * nb of releases (for each release we have 240 time points and for each of these points we have a longitude value)
            lat_argmax = spec_argmax['latitude']
            # let's see what's in lon_argmax, for example
            #lon_argmax
            # the data is not yet computed; in order to actually fire the computation (load data from netcdf and process it), we can call a method "load"; it will take a couple of seconds to execute:
            lon_argmax.load()
            lat_argmax.load()
            # and let's see what's in lon_argmax, for example:
            #lon_argmax
            spec_max = da.isel(longitude=lon_argmax, latitude=lat_argmax) 
            # again, we have to actually load spec_max to see results:
            spec_max.load()
            
            lon_argmax = lon_argmax.where(spec_max > 0)
            lat_argmax = lat_argmax.where(spec_max > 0)
            spec_max = spec_max.where(spec_max > 0)
            #spec_max
            
            da = da.assign_coords(pointspec=da.pointspec)
            
            #print("length of lon_argmax : ", len(lon_argmax))
            #print("length of lon_argmax[0] : ", len(lon_argmax[0]), "\n")
            
            ####################
            print(spec_argmax)
            ####################
            
            for i in range(len(lon_argmax)): #loop over all releases
                #if (lon_argmax[i].values)>0:
                #    print(lon_argmax[i].values)
                for j in range(len(lon_argmax[i])): #loop over all times
                    #print(lon_argmax[i,j].values)
                    for k in range (len(lon_argmax[i,j])): # loop over all heights I suppose ?
                        """ mais en fait là y a pas besoin de loop over all values si?
                        regarder juste max et min ça suffit je dirai """
                        if(lon_argmax[i,j,k].values)>=118 and (lon_argmax[i,j,k].values)<=298 and (lat_argmax[i,j,k].values)>=75 and (lat_argmax[i,j,k].values)<=284: # if lon between -120° and -30° (GOES 16)
                            # if lat between -52° and 52° (GOES 16)
                            #######
                            print(f"lon_argmax[{i},{j},{k}].values : {lon_argmax[i,j,k].values}, lat_argmax[{i},{j},{k}].values : {lat_argmax[i,j,k].values}")
                            #######
                            print("OK for further analysis using GOES 16 data")
                                    
                            return
                        
            print("Not suitable for further analysis at this time")        

In [10]:
# main programme
set_paths()
find_flights()

for i in range(len(flights)):
    i = 0 #################"""
    flight = flights[i]
    open_netcdf(flight)
    print("flight = ", flight)
    break ##################
    

['flight_2018_001_1h_05deg', 'flight_2018_003_1h_05deg', 'flight_2018_004_1h_05deg', 'flight_2018_005_1h_05deg', 'flight_2018_006_1h_05deg', 'flight_2018_007_1h_05deg', 'flight_2018_008_1h_05deg', 'flight_2018_010_1h_05deg', 'flight_2018_011_1h_05deg', 'flight_2018_012_1h_05deg', 'flight_2018_013_1h_05deg', 'flight_2018_014_1h_05deg', 'flight_2018_015_1h_05deg', 'flight_2018_016_1h_05deg', 'flight_2018_017_1h_05deg', 'flight_2018_018_1h_05deg', 'flight_2018_019_1h_05deg', 'flight_2018_020_1h_05deg', 'flight_2018_021_1h_05deg', 'flight_2018_022_1h_05deg', 'flight_2018_023_1h_05deg', 'flight_2018_024_1h_05deg', 'flight_2018_025_1h_05deg', 'flight_2018_026_1h_05deg', 'flight_2018_027_1h_05deg', 'flight_2018_028_1h_05deg', 'flight_2018_029_1h_05deg', 'flight_2018_030_1h_05deg']

Open file : /o3p/patj/SOFT-IO-LI/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/flight_2018_001_1h_05deg/10j_100k_output/grid_time_20180603150000.nc


/home/patj/miniconda3/envs/softioli/lib/python3.10/site-packages/xarray/core/variable.py:1722: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._stack_once(dims, new_dim)


{'longitude': <xarray.DataArray 'spec001_mr' (pointspec: 2, time: 240, height: 27)>
array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [317, 315, 281, ..., 265, 191,   6],
        [315, 315, 281, ..., 256, 191,  15],
        [315, 315, 317, ..., 256, 190,  13]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [220, 220, 220, ...,  75,  49, 709],
        [220, 223, 223, ...,  74,  46, 703],
        [224, 221, 223, ...,  72,  43, 695]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-06-03T14:00:00 ... 2018-05-24T15:00:00
  * height   (height) float32 500.0 1e+03 1.5e+03 ... 1.25e+04 1.3e+04 5e+04
Dimensions without coordinates: pointspec, 'latitude': <xarray.DataArray 'spec001_mr' (pointspec: 2, time: 240, height: 27)>
array([[[  0,   0,   0, ...,   0,   0,   0],
       